In [1]:
import timm
import torch
import torch.nn as nn
import torchvision
# import Dataloader
import torch.functional as F
import torch.optim as optim
import math
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import random_split
import torch.nn as nn
from tqdm import tqdm
# import Dataloader
# import Updated_DL
from torch.utils.data import random_split
from torch.utils.data import ConcatDataset 
# from sklearn.externals import joblib

from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import scipy.io
from PIL import Image
import pandas as pd
from torch.utils.data.dataset import random_split
import scipy
from timm.models.layers import GroupNormAct, ClassifierHead
from augmentation import *

In [2]:
# model = timm.create_model('resnetv2_50x1_bitm', pretrained=True) 
# model.stages[0]

In [3]:
class CUBDataset(torchvision.datasets.ImageFolder):
    """
    Dataset class for CUB Dataset
    """

    def __init__(self, image_root_path, caption_root_path=None, split="train", *args, **kwargs):
        """
        Args:
            image_root_path:      path to dir containing images and lists folders
            caption_root_path:    path to dir containing captions
            split:          train / test
            *args:
            **kwargs:
        """
        image_info = self.get_file_content(f"{image_root_path}/images.txt")
        self.image_id_to_name = {y[0]: y[1] for y in [x.strip().split(" ") for x in image_info]}
        split_info = self.get_file_content(f"{image_root_path}/train_test_split.txt")
        self.split_info = {self.image_id_to_name[y[0]]: y[1] for y in [x.strip().split(" ") for x in split_info]}
        self.split = "1" if split == "train" else "0"
        self.caption_root_path = caption_root_path

        super(CUBDataset, self).__init__(root=f"{image_root_path}/images", is_valid_file=self.is_valid_file,
                                         *args, **kwargs)

    def is_valid_file(self, x):
        return self.split_info[(x[len(self.root) + 1:])] == self.split

    @staticmethod
    def get_file_content(file_path):
        with open(file_path) as fo:
            content = fo.readlines()
        return content

In [4]:
class DOGDataset(torchvision.datasets.ImageFolder):
    """
    Dataset class for DOG Dataset
    """

    def __init__(self, image_root_path, caption_root_path=None, split="train", *args, **kwargs):
        """
        Args:
            image_root_path:      path to dir containing images and lists folders
            caption_root_path:    path to dir containing captions
            split:          train / test
            *args:
            **kwargs:
        """
        image_info = self.get_file_content(f"{image_root_path}/splits/file_list.mat")
        image_files = [o[0][0] for o in image_info]
        
        split_info = self.get_file_content(f"{image_root_path}/splits/{split}_list.mat")
        split_files = [o[0][0] for o in split_info]
        self.split_info = {}
        if split == 'train' :
            for image in image_files:
                if image in split_files:
                    self.split_info[image] = "1"
                else:
                    self.split_info[image] = "0"
        elif split== 'test' :
            for image in image_files:
                if image in split_files:
                    self.split_info[image] = "0"
                else:
                    self.split_info[image] = "1"
                    
        self.split = "1" if split == "train" else "0"
        self.caption_root_path = caption_root_path

        super(DOGDataset, self).__init__(root=f"{image_root_path}Images", is_valid_file = self.is_valid_file,
                                         *args, **kwargs)
        
        ## modify class index as we are going to concat to first dataset
        self.class_to_idx = {class_: idx+200 for idx, class_ in enumerate(self.class_to_idx)}
        
    def is_valid_file(self, x):
        return self.split_info[(x[len(self.root) + 1:])] == self.split
    
    def __getitem__(self, index):
        path, target = self.imgs[index]
        img = Image.open(os.path.join(path)).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        
        ## modify target class index as we are going to concat to first dataset
        return img, target + 200

    @staticmethod
    def get_file_content(file_path):
        content =  scipy.io.loadmat(file_path)
        return content['file_list']

In [5]:
data_root = "./CUB_200_2011"

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

# write data transform here as per the requirement
data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(0.1),
        transforms.RandomRotation(45),
        Shadows(),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])


train_dataset_cub = CUBDataset(image_root_path=f"{data_root}", transform=data_transform, split="train")
test_dataset_cub = CUBDataset(image_root_path=f"{data_root}", transform=test_transform, split="test")


# load in into the torch dataloader to get variable batch size, shuffle 
train_loader_cub = torch.utils.data.DataLoader(train_dataset_cub, batch_size=32, drop_last=True, shuffle=True)
test_loader_cub = torch.utils.data.DataLoader(test_dataset_cub, batch_size=32, drop_last=False, shuffle=False)

In [6]:
# Set train and test set
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(0.1),
        transforms.RandomRotation(45),
        Shadows(),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])


data_root = "./dog/dog/"

train_dataset_dog = DOGDataset(image_root_path=f"{data_root}", transform=data_transform, split="train")
test_dataset_dog = DOGDataset(image_root_path=f"{data_root}", transform=test_transform, split="test")
print('Number of train samples:', len(train_dataset_dog))
print('Number of test samples:', len(test_dataset_dog))

# load in into the torch dataloader to get variable batch size, shuffle 
train_loader_dog = torch.utils.data.DataLoader(train_dataset_dog, batch_size=32, drop_last=True, shuffle=True)
test_loader_dog = torch.utils.data.DataLoader(test_dataset_dog, batch_size=32, drop_last=False, shuffle=False)

Number of train samples: 12000
Number of test samples: 8580


In [7]:
concat_dataset_train = ConcatDataset([train_dataset_cub, train_dataset_dog])

t_split = int(len(concat_dataset_train) * 0.9)
v_split = len(concat_dataset_train) - t_split

concat_train_ds, concat_val_ds = random_split(concat_dataset_train, [t_split, v_split])

concat_loader_train = torch.utils.data.DataLoader(
             concat_train_ds,
             batch_size=10, shuffle=True,
             num_workers=1, pin_memory=True
            )

concat_loader_val = torch.utils.data.DataLoader(
             concat_val_ds,
             batch_size=10, shuffle=True,
             num_workers=1, pin_memory=True
            )

concat_dataset_test = ConcatDataset([test_dataset_cub, test_dataset_dog])

concat_loader_test = torch.utils.data.DataLoader(
             concat_dataset_test,
             batch_size=128, shuffle=False,
             num_workers=1, pin_memory=True
            )

In [8]:
class Downsampler(nn.Module):
    def __init__(self, planes, layer_out):
        super(Downsampler, self).__init__()
        # self.conv1 = nn.Conv2d(planes, planes, kernel_size=3, stride=2, padding=0)
        # self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride = 4, padding=0)
        # self.conv3 = nn.Conv2d(planes, planes, kernel_size=2, stride = 2, padding=0)
        self.maxpool1 = nn.MaxPool2d((8,8))
        self.maxpool2 = nn.MaxPool2d((4,4))
        self.maxpool3 = nn.MaxPool2d((2,2))
        self.layer_out = layer_out
        
        
    def forward(self, x):
        if self.layer_out == 1:

            #print('Input shape ', x.shape)
            x = self.maxpool1(x)
            #print('maxpool1 output shape ', x.shape)
            
        elif self.layer_out == 2:
            #print('Input shape ', x.shape)
            x = self.maxpool2(x)
            #print('maxpool2 output shape ', x.shape)
            
        elif self.layer_out == 3:
            #print('Input shape ', x.shape)
            x = self.maxpool3(x)
            #print('maxpool3 output shape ', x.shape)
            
        return x

In [9]:
class newModel(nn.Module):
    def __init__(self, pretrained, num_classes = 320):
        super().__init__()
        self.model = timm.create_model('resnetv2_50x1_bitm', pretrained=pretrained)
        
        for param in self.model.parameters():
            param.requires_grad = False
        
        self.down1 = Downsampler(64, layer_out=1)
        self.down2 = Downsampler(128, layer_out=2)
        self.down3 = Downsampler(256, layer_out=3)
        
        self.gnorm = GroupNormAct(2048+1024+512+256, num_groups=32)
        self.head = ClassifierHead(2048+1024+512+256, num_classes, pool_type='avg', use_conv=True)
        
    def forward(self, x):
        x = self.model.stem(x)
        x1 = self.model.stages[0](x)
        #print("X1", x1.shape)
        x2 = self.model.stages[1](x1)
        #print("X2", x2.shape)
        x3 = self.model.stages[2](x2)
        #print("X3", x3.shape)
        x = self.model.stages[3](x3)
        #print("X", x.shape)
        
        x1 = self.down1(x1)
        x2 = self.down2(x2)
        x3 = self.down3(x3)
        
        x = torch.cat((x1, x2, x3, x), dim=1)
        
        # x = self.model.norm(x)
        x = self.gnorm(x)
        x = self.head(x)
        return x
        

In [10]:
model = newModel(pretrained=True)

In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,236,800 trainable parameters


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else"cpu")
device

device(type='cuda')

In [13]:
model = model.to(device)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-4, betas = (0.75, 0.999), weight_decay = 1e-5)
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [15]:
# Train the model
num_epochs = 50
total_step = len(concat_loader_train)
curr_lr = 1e-4
# val_losses = np.array([])
# train_losses = np.array([])
val_losses = []
train_losses = []

val_acc = []
train_acc = []

myfile = open('myfile__resnet50_improved_txt', 'w')
myfile.write('Training results: \n')

for epoch in range(num_epochs):
    running_loss = 0
    correct=0
    total=0
    for i, (images, labels) in enumerate(concat_loader_train):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        # print(images.shape)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimizes
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        if (i+1) % 10 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Train Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    train_loss = running_loss / len(concat_loader_train)
    acc = 100.*correct/total

    train_losses.append(train_loss)
    train_acc.append(acc)
    print('Train loss at epoch {} is {:.4f}'.format(epoch+1, train_loss))
    print('Train acc at epoch {} is {:.4f}'.format(epoch+1, acc))
    myfile.write('Train loss at epoch {} is {:.4f}\n'.format(epoch+1, train_loss))
    myfile.write('Train acc at epoch {} is {:.4f}\n'.format(epoch+1, acc))

    model.eval()
    running_loss = 0
    with torch.no_grad():
        for i, (images, labels) in enumerate(concat_loader_val):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    val_loss = loss / len(concat_loader_val) 
    acc = 100.*correct/total
    val_losses.append(val_loss)
    val_acc.append(acc)
    print('Val loss at epoch {} is {:.4f}'.format(epoch+1, val_loss))
    print('Val acc at epoch {} is {:.4f}'.format(epoch+1, acc))
    myfile.write('Val loss at epoch {} is {:.4f}\n'.format(epoch+1, val_loss))
    myfile.write('Val acc at epoch {} is {:.4f}\n'.format(epoch+1, acc))    

    # Decay learning rate
    if (epoch+1) % 10 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)


myfile.close()

plt.plot(train_acc)
plt.plot(val_acc)
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Accuracy')

plt.show()


Epoch [1/50], Step [10/1620] Train Loss: 6.1899
Epoch [1/50], Step [20/1620] Train Loss: 5.8668
Epoch [1/50], Step [30/1620] Train Loss: 6.3852
Epoch [1/50], Step [40/1620] Train Loss: 5.9684
Epoch [1/50], Step [50/1620] Train Loss: 5.5636
Epoch [1/50], Step [60/1620] Train Loss: 5.8703
Epoch [1/50], Step [70/1620] Train Loss: 5.8201
Epoch [1/50], Step [80/1620] Train Loss: 5.1005
Epoch [1/50], Step [90/1620] Train Loss: 5.6768
Epoch [1/50], Step [100/1620] Train Loss: 6.0024
Epoch [1/50], Step [110/1620] Train Loss: 5.1925
Epoch [1/50], Step [120/1620] Train Loss: 5.2130
Epoch [1/50], Step [130/1620] Train Loss: 6.1043
Epoch [1/50], Step [140/1620] Train Loss: 5.5538
Epoch [1/50], Step [150/1620] Train Loss: 5.9343
Epoch [1/50], Step [160/1620] Train Loss: 5.6224
Epoch [1/50], Step [170/1620] Train Loss: 5.7082
Epoch [1/50], Step [180/1620] Train Loss: 5.7166
Epoch [1/50], Step [190/1620] Train Loss: 5.2945
Epoch [1/50], Step [200/1620] Train Loss: 5.5060
Epoch [1/50], Step [210/1620]

Epoch [2/50], Step [30/1620] Train Loss: 4.1312
Epoch [2/50], Step [40/1620] Train Loss: 4.0058
Epoch [2/50], Step [50/1620] Train Loss: 3.0977
Epoch [2/50], Step [60/1620] Train Loss: 4.2278
Epoch [2/50], Step [70/1620] Train Loss: 3.3423
Epoch [2/50], Step [80/1620] Train Loss: 4.0391
Epoch [2/50], Step [90/1620] Train Loss: 4.2522
Epoch [2/50], Step [100/1620] Train Loss: 3.3071
Epoch [2/50], Step [110/1620] Train Loss: 3.4827
Epoch [2/50], Step [120/1620] Train Loss: 3.4168
Epoch [2/50], Step [130/1620] Train Loss: 4.1064
Epoch [2/50], Step [140/1620] Train Loss: 4.1891
Epoch [2/50], Step [150/1620] Train Loss: 3.7445
Epoch [2/50], Step [160/1620] Train Loss: 4.1133
Epoch [2/50], Step [170/1620] Train Loss: 3.3498
Epoch [2/50], Step [180/1620] Train Loss: 4.1023
Epoch [2/50], Step [190/1620] Train Loss: 3.8108
Epoch [2/50], Step [200/1620] Train Loss: 3.8763
Epoch [2/50], Step [210/1620] Train Loss: 3.4191
Epoch [2/50], Step [220/1620] Train Loss: 4.1137
Epoch [2/50], Step [230/162

Epoch [3/50], Step [50/1620] Train Loss: 3.0434
Epoch [3/50], Step [60/1620] Train Loss: 2.4727
Epoch [3/50], Step [70/1620] Train Loss: 3.4205
Epoch [3/50], Step [80/1620] Train Loss: 3.7384
Epoch [3/50], Step [90/1620] Train Loss: 3.0402
Epoch [3/50], Step [100/1620] Train Loss: 3.4328
Epoch [3/50], Step [110/1620] Train Loss: 3.1960
Epoch [3/50], Step [120/1620] Train Loss: 3.0036
Epoch [3/50], Step [130/1620] Train Loss: 3.2107
Epoch [3/50], Step [140/1620] Train Loss: 3.0869
Epoch [3/50], Step [150/1620] Train Loss: 3.3267
Epoch [3/50], Step [160/1620] Train Loss: 3.5939
Epoch [3/50], Step [170/1620] Train Loss: 3.2043
Epoch [3/50], Step [180/1620] Train Loss: 3.4090
Epoch [3/50], Step [190/1620] Train Loss: 3.7243
Epoch [3/50], Step [200/1620] Train Loss: 3.0215
Epoch [3/50], Step [210/1620] Train Loss: 2.7040
Epoch [3/50], Step [220/1620] Train Loss: 4.4860
Epoch [3/50], Step [230/1620] Train Loss: 3.3145
Epoch [3/50], Step [240/1620] Train Loss: 2.3608
Epoch [3/50], Step [250/1

Epoch [4/50], Step [70/1620] Train Loss: 2.6613
Epoch [4/50], Step [80/1620] Train Loss: 2.4330
Epoch [4/50], Step [90/1620] Train Loss: 1.8739
Epoch [4/50], Step [100/1620] Train Loss: 1.6474
Epoch [4/50], Step [110/1620] Train Loss: 2.6200
Epoch [4/50], Step [120/1620] Train Loss: 2.5258
Epoch [4/50], Step [130/1620] Train Loss: 2.1614
Epoch [4/50], Step [140/1620] Train Loss: 2.6591
Epoch [4/50], Step [150/1620] Train Loss: 2.6502
Epoch [4/50], Step [160/1620] Train Loss: 2.8747
Epoch [4/50], Step [170/1620] Train Loss: 2.7905
Epoch [4/50], Step [180/1620] Train Loss: 3.0634
Epoch [4/50], Step [190/1620] Train Loss: 1.8275
Epoch [4/50], Step [200/1620] Train Loss: 3.3054
Epoch [4/50], Step [210/1620] Train Loss: 3.4541
Epoch [4/50], Step [220/1620] Train Loss: 3.4208
Epoch [4/50], Step [230/1620] Train Loss: 3.0723
Epoch [4/50], Step [240/1620] Train Loss: 1.6573
Epoch [4/50], Step [250/1620] Train Loss: 2.3078
Epoch [4/50], Step [260/1620] Train Loss: 3.1710
Epoch [4/50], Step [270

Epoch [5/50], Step [90/1620] Train Loss: 1.8918
Epoch [5/50], Step [100/1620] Train Loss: 2.2973
Epoch [5/50], Step [110/1620] Train Loss: 2.5244
Epoch [5/50], Step [120/1620] Train Loss: 2.9280
Epoch [5/50], Step [130/1620] Train Loss: 2.8002
Epoch [5/50], Step [140/1620] Train Loss: 2.4232
Epoch [5/50], Step [150/1620] Train Loss: 2.6340
Epoch [5/50], Step [160/1620] Train Loss: 1.1559
Epoch [5/50], Step [170/1620] Train Loss: 2.0895
Epoch [5/50], Step [180/1620] Train Loss: 2.6555
Epoch [5/50], Step [190/1620] Train Loss: 2.2437
Epoch [5/50], Step [200/1620] Train Loss: 2.3401
Epoch [5/50], Step [210/1620] Train Loss: 2.5934
Epoch [5/50], Step [220/1620] Train Loss: 1.2724
Epoch [5/50], Step [230/1620] Train Loss: 2.5797
Epoch [5/50], Step [240/1620] Train Loss: 1.9469
Epoch [5/50], Step [250/1620] Train Loss: 3.2143
Epoch [5/50], Step [260/1620] Train Loss: 2.2123
Epoch [5/50], Step [270/1620] Train Loss: 2.4280
Epoch [5/50], Step [280/1620] Train Loss: 3.6244
Epoch [5/50], Step [2

...

Epoch [27/50], Step [20/1620] Train Loss: 1.9854
Epoch [27/50], Step [30/1620] Train Loss: 1.9371
Epoch [27/50], Step [40/1620] Train Loss: 2.0821
Epoch [27/50], Step [50/1620] Train Loss: 1.3751
Epoch [27/50], Step [60/1620] Train Loss: 1.9938
Epoch [27/50], Step [70/1620] Train Loss: 0.7853
Epoch [27/50], Step [80/1620] Train Loss: 1.3878
Epoch [27/50], Step [90/1620] Train Loss: 0.8404
Epoch [27/50], Step [100/1620] Train Loss: 1.0484
Epoch [27/50], Step [110/1620] Train Loss: 2.0386
Epoch [27/50], Step [120/1620] Train Loss: 1.1701
Epoch [27/50], Step [130/1620] Train Loss: 0.9277
Epoch [27/50], Step [140/1620] Train Loss: 1.4826
Epoch [27/50], Step [150/1620] Train Loss: 0.8408
Epoch [27/50], Step [160/1620] Train Loss: 2.3066
Epoch [27/50], Step [170/1620] Train Loss: 1.4402
Epoch [27/50], Step [180/1620] Train Loss: 0.7706
Epoch [27/50], Step [190/1620] Train Loss: 1.3027
Epoch [27/50], Step [200/1620] Train Loss: 1.9381
Epoch [27/50], Step [210/1620] Train Loss: 1.1953
Epoch [2

Epoch [28/50], Step [10/1620] Train Loss: 1.2006
Epoch [28/50], Step [20/1620] Train Loss: 2.5510
Epoch [28/50], Step [30/1620] Train Loss: 1.5008
Epoch [28/50], Step [40/1620] Train Loss: 1.9345
Epoch [28/50], Step [50/1620] Train Loss: 0.6984
Epoch [28/50], Step [60/1620] Train Loss: 2.3628
Epoch [28/50], Step [70/1620] Train Loss: 1.0185
Epoch [28/50], Step [80/1620] Train Loss: 1.4047
Epoch [28/50], Step [90/1620] Train Loss: 1.8248
Epoch [28/50], Step [100/1620] Train Loss: 0.8949
Epoch [28/50], Step [110/1620] Train Loss: 1.2746
Epoch [28/50], Step [120/1620] Train Loss: 0.9367
Epoch [28/50], Step [130/1620] Train Loss: 1.0305
Epoch [28/50], Step [140/1620] Train Loss: 1.5779
Epoch [28/50], Step [150/1620] Train Loss: 1.6202
Epoch [28/50], Step [160/1620] Train Loss: 1.2345
Epoch [28/50], Step [170/1620] Train Loss: 1.6466
Epoch [28/50], Step [180/1620] Train Loss: 1.1125
Epoch [28/50], Step [190/1620] Train Loss: 1.6821
Epoch [28/50], Step [200/1620] Train Loss: 1.7993
Epoch [28

Val loss at epoch 28 is 0.0064
Val acc at epoch 28 is 59.3920
Epoch [29/50], Step [10/1620] Train Loss: 0.9197
Epoch [29/50], Step [20/1620] Train Loss: 1.8243
Epoch [29/50], Step [30/1620] Train Loss: 2.1539
Epoch [29/50], Step [40/1620] Train Loss: 0.9099
Epoch [29/50], Step [50/1620] Train Loss: 1.5334
Epoch [29/50], Step [60/1620] Train Loss: 1.3564
Epoch [29/50], Step [70/1620] Train Loss: 0.8873
Epoch [29/50], Step [80/1620] Train Loss: 1.4317
Epoch [29/50], Step [90/1620] Train Loss: 1.1379
Epoch [29/50], Step [100/1620] Train Loss: 2.2190
Epoch [29/50], Step [110/1620] Train Loss: 1.7877
Epoch [29/50], Step [120/1620] Train Loss: 1.1231
Epoch [29/50], Step [130/1620] Train Loss: 1.1918
Epoch [29/50], Step [140/1620] Train Loss: 1.0162
Epoch [29/50], Step [150/1620] Train Loss: 2.3961
Epoch [29/50], Step [160/1620] Train Loss: 2.2631
Epoch [29/50], Step [170/1620] Train Loss: 1.3642
Epoch [29/50], Step [180/1620] Train Loss: 2.1995
Epoch [29/50], Step [190/1620] Train Loss: 2.05

Val loss at epoch 29 is 0.0103
Val acc at epoch 29 is 59.6477
Epoch [30/50], Step [10/1620] Train Loss: 0.5736
Epoch [30/50], Step [20/1620] Train Loss: 1.4703
Epoch [30/50], Step [30/1620] Train Loss: 1.9148
Epoch [30/50], Step [40/1620] Train Loss: 2.1767
Epoch [30/50], Step [50/1620] Train Loss: 1.5672
Epoch [30/50], Step [60/1620] Train Loss: 2.0549
Epoch [30/50], Step [70/1620] Train Loss: 1.9145
Epoch [30/50], Step [80/1620] Train Loss: 2.8396
Epoch [30/50], Step [90/1620] Train Loss: 1.1604
Epoch [30/50], Step [100/1620] Train Loss: 0.8681
Epoch [30/50], Step [110/1620] Train Loss: 1.8086
Epoch [30/50], Step [120/1620] Train Loss: 1.4336
Epoch [30/50], Step [130/1620] Train Loss: 1.2059
Epoch [30/50], Step [140/1620] Train Loss: 1.5412
Epoch [30/50], Step [150/1620] Train Loss: 1.4672
Epoch [30/50], Step [160/1620] Train Loss: 1.3954
Epoch [30/50], Step [170/1620] Train Loss: 1.4566
Epoch [30/50], Step [180/1620] Train Loss: 1.4873
Epoch [30/50], Step [190/1620] Train Loss: 1.35

KeyboardInterrupt: 

In [ ]:
plt.plot(train_acc)
plt.plot(val_acc)
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train','Valid'])
plt.title('Train vs Valid Accuracy')

plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for imgs, lbls in concat_loader_val:
        imgs = imgs.to(device)
        lbls = lbls.to(device)
        outputs = model(imgs)
        _, predicted = torch.max(outputs.data, 1)
        total += lbls.size(0)
        correct += (predicted == lbls).sum().item()
        
        
        
    print('Accuracy of the model on the test images : {} %'.format(100 * correct / total))

In [ ]:
x1 = torch.randn([1, 3, 224, 224])

In [ ]:
img = torch.randn([1, 3, 224, 224])
model = newModel(pretrained=True)
output = model(img)
output.shape

In [ ]:
out1 = torch.randn([1, 128, 56, 56])
out2 = torch.randn([1, 256, 28, 28])
out3 = torch.randn([1, 512, 14, 14])
down1 = Downsampler(out1.shape[1], layer_out = 1)
down2 = Downsampler(out2.shape[1], layer_out = 2)
down3 = Downsampler(out3.shape[1], layer_out = 3)

out_down1 = down1(out1)
print(out_down1.shape, '\n')
out_down2 = down2(out2)
print(out_down2.shape, '\n')
out_down3 = down3(out3)
print(out_down3.shape)